<a href="https://colab.research.google.com/github/samueleallen/Scraping-Val-Data/blob/main/scraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import os # maybe not needed
import pandas as pd

# Step 1: scrape team urls from standings page
standings_url = 'https://www.vlr.gg/vct-2024/standings'
data = requests.get(standings_url)
from bs4 import BeautifulSoup
soup = BeautifulSoup(data.text)

# Locate team urls
standings_table = soup.select('div.eg-standing-container')[0]
links = standings_table.find_all('a')
links = [l.get('href') for l in links]
links = [l for l in links if '/team/' in l]
# Format link
team_urls = [f'https://www.vlr.gg{l}' for l in links]
team_urls = team_urls[0]
data = requests.get(team_urls)

# Step 2: Scrape stats section data
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'team/stats' in l]
# Successful, links = ['/team/stats/2359/leviat-n/']
data = requests.get(f"https://www.vlr.gg{links[0]}")
soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
table = soup.find('table', class_='wf-table mod-team-maps')

# Use pandas to read the table
stats = pd.read_html(str(table))[0]

# Step 5: Filter rows that start with specific map names, kind of ruins table
map_names = ["Sunset", "Bind", "Haven", "Split", "Ascent", "Icebox", "Breeze", "Fracture", "Pearl", "Lotus", "Abyss"]
filtered_stats = stats[stats.iloc[:, 0].str.startswith(tuple(map_names), na=False)]

# Display the filtered table
# print(filtered_stats)

# Individual Player Stats
data = requests.get(team_urls)
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'player' in l]
# Links provides players + staff
player_urls = [f"https://www.vlr.gg{l}" for l in links]

# Format player stat links to entire career
player_stats = [f"{l}/?timespan=all" for l in player_urls]

# Keep only players, not interested in coaches + staff
player_stats = player_stats[:5]
data = requests.get(player_stats[0])
soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
table = soup.find('table', class_='wf-table')

# Use pandas to read the table
player_stats = pd.read_html(str(table))[0]
# Table has an error in first column where it can't read agent names since they are images on website
agent_names = []
for img_tag in table.find_all('img'):
    img_src = img_tag.get('src') # get image sources / agent names
    agent_name = os.path.splitext(os.path.basename(img_src))[0]
    agent_names.append(agent_name)

player_stats['Agent'] = agent_names  # Add a new 'Agent' column
#player_stats.head()
# now we have a new agent column that is fully functioning!

# Repeat player stats scraper but for recent 90 days to understand recent performances
player_stats = [f"{l}/?timespan=90d" for l in player_urls]

# Keep only players, not interested in coaches + staff
player_stats = player_stats[:5]
data = requests.get(player_stats[0])
soup = BeautifulSoup(data.text)  # Create BeautifulSoup object for stats page
table = soup.find('table', class_='wf-table')

# Use pandas to read the table
player_stats = pd.read_html(str(table))[0]
# Table has an error in first column where it can't read agent names since they are images on website
agent_names = []
for img_tag in table.find_all('img'):
    img_src = img_tag.get('src') # get image sources / agent names
    agent_name = os.path.splitext(os.path.basename(img_src))[0]
    agent_names.append(agent_name)

player_stats['Agent'] = agent_names  # Add a new 'Agent' column

# I have all-time player stats, past 90 days player stats, team map stats, now we need match up stats for teams. Ex: fnatic vs sentinels, sentinels has won x out of y matchups
matches_url = 'https://www.vlr.gg/matches'
data = requests.get(matches_url)
soup = BeautifulSoup(data.text)
links = soup.find_all('a')
links = [l.get("href") for l in links]
links = [l for l in links if l and 'page' in l]
url = "https://www.vlr.gg/matches/results"  # Replace with the actual URL
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')

# Find all page number links
page_links = soup.find_all("a", class_="btn mod-page")
# Page_links holds a list of HTML <a> elements beautiful soup found
# Now extract the page numbers
page_numbers = [int(link.text) for link in page_links if link.text.isdigit()]

# Get the max page number
max_page = max(page_numbers)

for page in range(1, max_page+1): # For loop starts from page 1 ends at max page
    url = f"https://www.vlr.gg/matches/results/?page={page}"
    data = requests.get(url)
    soup = BeautifulSoup(data.text, "html.parser")
    links = soup.find_all('a')
    links = [l.get("href") for l in links]
    # Filter links to only include recent years, champions tour matches, and not acension champions tour matches
    links = [l for l in links if l and 'champions-tour' in l and ('2023' in l or '2024' in l or '2025' in l) and 'ascension' not in l and 'challengers' not in l]
    matches = [f"https://www.vlr.gg{l}" for l in links]
    print(matches[0])


<ipython-input-3-e0f09dccea96>:32: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  stats = pd.read_html(str(table))[0]
<ipython-input-3-e0f09dccea96>:60: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]
<ipython-input-3-e0f09dccea96>:82: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  player_stats = pd.read_html(str(table))[0]


https://www.vlr.gg/430855/jdg-esports-vs-xi-lai-gaming-champions-tour-2025-china-kickoff-lr2
https://www.vlr.gg/430853/xi-lai-gaming-vs-all-gamers-champions-tour-2025-china-kickoff-lr1
https://www.vlr.gg/430840/all-gamers-vs-nova-esports-champions-tour-2025-china-kickoff-ur1


IndexError: list index out of range